In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [ ]:
train_org=pd.read_csv("train_m.csv",sep=',')

In [ ]:
test_org=pd.read_csv("holdout_features.csv",sep=',')

In [ ]:
#print(test_org.loc[test_org['asin'] == 'B07N8H836C'])
#B08WJPQNYB
#print(test_org.loc[test_org['asin'] == 'B08WJPQNYB'])

In [ ]:
train_org.head(3)

,asin,market,sales_first_seen,total_unit_sold,product_company_type_source,retailer,product_category,product_brand,product_name,sku,...,review_body,reviewer_location,product_description,product_tags,product_contents,manufacturer,brand,product_group,category,subcategory
0,B01MR5SH9Z,GB,27-12-2020,22353,Competitor,amazon.co.uk,Dental Care > Toothpastes,Biomed,Biomed Sensitive Natural Toothpaste for Enamel...,B01MR5SH9Z,...,Not sure about the whitening capability but I ...,GB,Biomed sensitive natural toothpaste is free fr...,Biomed Sensitive Natural Toothpaste for Enamel...,"Aqua, Hydrated Silica, Hydrogenated Starch Hyd...",SPLAT,SPLAT,PERSONAL CARE,ORAL CARE,TOOTHPASTE
1,B01MR5SH9Z,GB,27-12-2020,22353,Competitor,amazon.co.uk,Dental Care > Toothpastes,Biomed,Biomed Sensitive Natural Toothpaste for Enamel...,B01MR5SH9Z,...,Says &quotmade in [LOCATION]&quot. It's a disg...,GB,Biomed sensitive natural toothpaste is free fr...,Biomed Sensitive Natural Toothpaste for Enamel...,"Aqua, Hydrated Silica, Hydrogenated Starch Hyd...",SPLAT,SPLAT,PERSONAL CARE,ORAL CARE,TOOTHPASTE
2,B01MR5SH9Z,GB,27-12-2020,22353,Competitor,amazon.co.uk,Dental Care > Toothpastes,Biomed,Biomed Sensitive Natural Toothpaste for Enamel...,B01MR5SH9Z,...,I wanted fluoride free toothpaste ..these seem...,GB,Biomed sensitive natural toothpaste is free fr...,Biomed Sensitive Natural Toothpaste for Enamel...,"Aqua, Hydrated Silica, Hydrogenated Starch Hyd...",SPLAT,SPLAT,PERSONAL CARE,ORAL CARE,TOOTHPASTE


In [ ]:
test_org.head(3)

,asin,market,sales_first_seen,product_company_type_source,retailer,product_category,product_brand,product_name,sku,review_timestamp,...,review_body,reviewer_location,product_description,product_tags,product_contents,manufacturer,brand,product_group,category,subcategory
0,B08WJPQNYB,US,27/12/2020,Competitor,amazon.com,Home & Kitchen > Home Décor Products > Home Fr...,Poo-Pourri,"Poo-Pourri Before-You- go Toilet Spray, 2 Oz, ...",B08WJPQNYB,30/04/2021 00:00,...,"Me gusta el producto, pero detesto que no es w...",US,Wave hello to this fresh new scent! Fresh Sea ...,"Poo-Pourri Before-You- go Toilet Spray, 2 Oz, ...",Make sure this fits by entering your model num...,POO-POURRI,POO-POURRI,HOME CARE,LAUNDRY,FABRIC CONDITIONER
1,B08WJPQNYB,US,27/12/2020,Competitor,amazon.com,Home & Kitchen > Home Décor Products > Home Fr...,Poo-Pourri,"Poo-Pourri Before-You- go Toilet Spray, 2 Oz, ...",B08WJPQNYB,24/04/2021 00:00,...,This product is one of the greatest inventions.,US,Wave hello to this fresh new scent! Fresh Sea ...,"Poo-Pourri Before-You- go Toilet Spray, 2 Oz, ...",Make sure this fits by entering your model num...,POO-POURRI,POO-POURRI,HOME CARE,LAUNDRY,FABRIC CONDITIONER
2,B08WJPQNYB,US,27/12/2020,Competitor,amazon.com,Home & Kitchen > Home Décor Products > Home Fr...,Poo-Pourri,"Poo-Pourri Before-You- go Toilet Spray, 2 Oz, ...",B08WJPQNYB,24/04/2021 00:00,...,Does what it claims. 🌸🌸🌸 I just wish it would ...,US,Wave hello to this fresh new scent! Fresh Sea ...,"Poo-Pourri Before-You- go Toilet Spray, 2 Oz, ...",Make sure this fits by entering your model num...,POO-POURRI,POO-POURRI,HOME CARE,LAUNDRY,FABRIC CONDITIONER


In [ ]:
print(train_org.columns)

Index(['asin', 'market', 'sales_first_seen', 'total_unit_sold',
       'product_company_type_source', 'retailer', 'product_category',
       'product_brand', 'product_name', 'sku', 'review_timestamp',
       'review_rating', 'review_title', 'review_body', 'reviewer_location',
       'product_description', 'product_tags', 'product_contents',
       'manufacturer', 'brand', 'product_group', 'category', 'subcategory'],
      dtype='object')


In [ ]:
filt_df=train_org[['asin','market','total_unit_sold','product_company_type_source','retailer','category','review_rating']]
filt_test_df=test_org[['asin','market','product_company_type_source','retailer','category','review_rating']]

In [ ]:
filt_df.head(3)

,asin,market,total_unit_sold,product_company_type_source,retailer,category,review_rating
0,B01MR5SH9Z,GB,22353,Competitor,amazon.co.uk,ORAL CARE,5
1,B01MR5SH9Z,GB,22353,Competitor,amazon.co.uk,ORAL CARE,1
2,B01MR5SH9Z,GB,22353,Competitor,amazon.co.uk,ORAL CARE,5


In [ ]:
print(filt_test_df.loc[filt_test_df['asin'] == 'B07N8H836C'])

            asin market product_company_type_source      retailer    category  \
1400  B07N8H836C     GB                  Competitor  amazon.co.uk  TOOTHPASTE   
1401  B07N8H836C     GB                  Competitor  amazon.co.uk  TOOTHPASTE   
1402  B07N8H836C     GB                  Competitor  amazon.co.uk  TOOTHPASTE   
1403  B07N8H836C     GB                  Competitor  amazon.co.uk  TOOTHPASTE   
1404  B07N8H836C     GB                  Competitor  amazon.co.uk  TOOTHPASTE   
1405  B07N8H836C     GB                  Competitor  amazon.co.uk  TOOTHPASTE   
1406  B07N8H836C     GB                  Competitor  amazon.co.uk  TOOTHPASTE   
1407  B07N8H836C     GB                  Competitor  amazon.co.uk  TOOTHPASTE   
1408  B07N8H836C     GB                  Competitor  amazon.co.uk  TOOTHPASTE   
1409  B07N8H836C     GB                  Competitor  amazon.co.uk  TOOTHPASTE   
1410  B07N8H836C     GB                  Competitor  amazon.co.uk  TOOTHPASTE   
1411  B07N8H836C     GB     

In [ ]:
avg_rating=filt_df.groupby(['asin','market','product_company_type_source','retailer','category','total_unit_sold'],as_index=False)[('review_rating')].mean()
avg_rating=pd.DataFrame(avg_rating,columns=['asin','market','product_company_type_source','retailer','category','total_unit_sold','review_rating'])


In [ ]:
avg_rating_predict=filt_test_df.groupby(['asin','market','product_company_type_source','retailer','category'],as_index=False)[('review_rating')].mean()

In [ ]:
print(len(avg_rating_predict))

164


In [ ]:
#print(avg_rating_predict[['asin'=='B07N8H836C']])
print(avg_rating_predict.loc[avg_rating_predict['asin'] == 'B07N8H836C'])

          asin market product_company_type_source      retailer    category  \
31  B07N8H836C     GB                  Competitor  amazon.co.uk  TOOTHPASTE   

    review_rating  
31           4.56  


In [ ]:
train=avg_rating.copy()
test=avg_rating_predict.copy()

In [ ]:
train.head(3)

,asin,market,product_company_type_source,retailer,category,total_unit_sold,review_rating
0,B01MR5SH9Z,GB,Competitor,amazon.co.uk,ORAL CARE,22353,3.64
1,B08G9ZJMZ7,US,Competitor,amazon.com,LAUNDRY,620,4.08
2,B08H2QWGWK,US,Competitor,amazon.com,TOTAL TEA,22776,4.20


In [ ]:
test.head(3)

,asin,market,product_company_type_source,retailer,category,review_rating
0,B000GKD5TW,US,Competitor,amazon.com,HOUSEHOLD CLEANERS,4.64
1,B000O6EFL0,US,Competitor,amazon.com,MEAL MAKERS,3.84
2,B0012YQ0X4,GB,Competitor,amazon.co.uk,SKIN CLEANSING,4.12


In [ ]:
#Map features and Target variables for the LGB model
target=['total_unit_sold']
#features=['asin','market','product_company_type_source','retailer','category','subcategory','review_rating']
#cat_features=['asin','market','product_company_type_source','retailer','category','subcategory']
features=['asin','market','product_company_type_source','retailer','category','review_rating']
cat_features=['asin','market','product_company_type_source','retailer','category']



#Define Categorical values for non-quantitative data
label_encoder_asin_train = LabelEncoder()
label_encoder_market_train = LabelEncoder()
label_encoder_product_company_type_source_train = LabelEncoder()
label_encoder_retailer_train = LabelEncoder()
label_encoder_category_train = LabelEncoder()
#label_encoder_subcategory_train = LabelEncoder()

train['asin'] = label_encoder_asin_train.fit_transform(train['asin'])
train['market'] = label_encoder_market_train.fit_transform(train['market'])
train['product_company_type_source'] = label_encoder_product_company_type_source_train.fit_transform(train['product_company_type_source'])
train['retailer'] = label_encoder_retailer_train.fit_transform(train['retailer'])
train['category'] = label_encoder_category_train.fit_transform(train['category'])
#train['subcategory'] = label_encoder_subcategory_train.fit_transform(train['subcategory'])

label_encoder_test_asin = LabelEncoder()
label_encoder_test_market = LabelEncoder()
label_encoder_test_product_company_type_source = LabelEncoder()
label_encoder_test_retailer = LabelEncoder()
label_encoder_test_category = LabelEncoder()
#label_encoder_test_subcategory = LabelEncoder()

test['asin'] = label_encoder_test_asin.fit_transform(test['asin'])
test['market'] = label_encoder_test_market.fit_transform(test['market'])
test['product_company_type_source'] = label_encoder_test_product_company_type_source.fit_transform(test['product_company_type_source'])
test['retailer'] = label_encoder_test_retailer.fit_transform(test['retailer'])
test['category'] = label_encoder_test_category.fit_transform(test['category'])
#test['subcategory'] = label_encoder_test_subcategory.fit_transform(test['subcategory'])




#Split data into training and validation dataset
dataset1=train.iloc[2:,:]
dataset2=train.iloc[:2,:]
# print(dataset1)
# print(dataset2)
x_train = dataset1[features].fillna(value=0)
y_train = dataset1[target].fillna(value=0)
x_valid=dataset2[features].fillna(value=0)
y_valid=dataset2[target].fillna(value=0)
dtrain=lgb.Dataset(x_train,y_train)
dvalid=lgb.Dataset(x_valid,y_valid)

In [ ]:
dataset2.head(20)

,asin,market,product_company_type_source,retailer,category,total_unit_sold,review_rating
0,0,0,0,0,3,22353,3.64
1,1,1,0,1,2,620,4.08


In [ ]:
dataset1.head(20)

,asin,market,product_company_type_source,retailer,category,total_unit_sold,review_rating
2,2,1,0,1,4,22776,4.20
3,3,1,0,1,1,9507,3.96
4,4,1,0,1,0,110494,3.96


In [ ]:
#Define parameters for the LGB Model
params = {'metric': 'rmse',
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise' : True,
          'random_state': 10}
 
#Train the Boosting based ensemble model
lgb_model=lgb.train(params=params,
                    train_set=dtrain,
                    num_boost_round=1500,
                    valid_sets=(dtrain, dvalid),
                    early_stopping_rounds=150,
                    categorical_feature=cat_features,
                    verbose_eval=100)   
 

Training until validation scores don't improve for 150 rounds.
[100]	training's rmse: 65365.8	valid_1's rmse: 84400.6
Early stopping, best iteration is:
[1]	training's rmse: 65365.8	valid_1's rmse: 84400.6


/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['asin', 'category', 'market', 'product_company_type_source', 'retailer']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


In [ ]:
#Predict the power generated output variable
test1=test[features]
test1=test1.fillna(0)
test1['preds']=lgb_model.predict(test1)


In [ ]:
test1.head(10)

,asin,market,product_company_type_source,retailer,category,review_rating,preds
0,0,2,0,2,13,4.64,47592.333333
1,1,2,0,2,17,3.84,47592.333333
2,2,1,0,1,24,4.12,47592.333333
3,3,1,0,1,23,4.52,47592.333333
4,4,2,0,2,11,3.16,47592.333333
5,5,2,0,2,21,4.88,47592.333333
6,6,2,0,2,7,4.44,47592.333333
7,7,2,0,2,21,3.72,47592.333333
8,8,2,0,2,13,4.28,47592.333333
9,9,2,0,2,7,4.32,47592.333333


In [ ]:
preds_1=test1[['asin','market','preds']]
preds_1.columns=['asin','market','preds']
preds_1['asin']=label_encoder_test_asin.inverse_transform(preds_1['asin'])
preds_1['market']=label_encoder_test_market.inverse_transform(preds_1['market'])
 
#Export the predicted values to a CSV for final submission
preds_1.to_csv('my_submission.csv',index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
preds_1.head(10)

,asin,market,preds
0,B000GKD5TW,US,47592.333333
1,B000O6EFL0,US,47592.333333
2,B0012YQ0X4,GB,47592.333333
3,B0013GAQQI,GB,47592.333333
4,B001MS7GZG,US,47592.333333
5,B002697YLG,US,47592.333333
6,B0045KBP9Q,US,47592.333333
7,B006CFVXSU,US,47592.333333
8,B008G03YBS,US,47592.333333
9,B008HD9KOO,US,47592.333333


In [ ]:
print(len(preds_1))

164
